# Classificação de artigos de moda com ML e Fashion MNIST

![](https://github.com/asantos2000/ml-ipt-hw/raw/master/pics/fashion_mnist_dataset_sample.png)

Figure 1. Fashion-MNIST samples (by Zalando, MIT License)

## 1. Problema

> Em 2021, as receitas de comércio eletrônico de varejo das vendas de vestuário e acessórios nos Estados Unidos totalizaram 180,5 bilhões de dólares, aumentando de 144,8 bilhões em 2020. - [Stadist.com](https://www.statista.com/statistics/278890/us-apparel-and-accessories-retail-e-commerce-revenue/)

Um grande problema que este mercado enfrenta é categorizar essas roupas e acessórios apenas pelas imagens, especialmente quando as categorias fornecidas pelas marcas são inconsistentes.

Os clientes não reconhecem as categorias dos produtos que estão buscando e desistem após a primeira pesquisa ou navegação pelas categorias. Cerca de 15% dos usuários desistem na funcionalidade de busca e 30% quando usam o seletor de categorias. Com o uso de reconhecimento por imagem, deseja-se reduzir em 10% o número de desistências em seis meses.

No armazém, estoquistas atribuem 15% das peças a categorias incorretas. Deseja-se reduzir em 10% os erros de atribuição.

Deseja-se identificar, a partir de uma imagem provida pelo usuário, em um dispositivo móvel, a qual categoria aquele item pertence.

### 1.1 Problema de ML

O objetivo é classificar as imagens fornecidas em dez categorias (classificação múltipla) utilizando para treinamento o dataset Fashion MNist e obter uma taxa de acerto acima de 90%.

Os seguintes modelos serão avaliados:

- Aprendizado por transferência: Com os modelos ResNet152 V2, VGG-16, DenseNet169;
- SVM
- ConvNet

Este trabalho está organizado da seguinte forma: 1. Discussão sobre o problema, 2. Planejamento do experimento, 3. Execução do experimento, 4. Análise e interpretação e Apresentação dos resultados.

## 2. Condução do experimento

### 2.1 Pipeline

![](https://github.com/asantos2000/ml-ipt-hw/raw/master/pics/ml-pipeline.png)

## Bibliotecas

In [ ]:
#! pip install -r requirements.txt

In [ ]:
# Only for kaggle
from shutil import copyfile

# upload python module as a data (file | upload data)
# copy our file into the working directory (make sure it has .py suffix)
copyfile(src = "../input/module/mod_util.py", dst = "../working/mod_util.py")

In [ ]:
# Import important libraries
import time
import wandb
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from mod_util import *

In [ ]:
# Globais
df_model_metrics = pd.DataFrame()

In [ ]:
# Coleta de métrias de hardware
os.environ["WANDB_NOTEBOOK_NAME"] = "ml-fashion-mnist-classification.ipynb"
#os.environ["WANDB_API_KEY"] = "key"
wandb.init(project="transfer-learning")

### 2.2 Descrição do conjunto de dados

Fashion-MNIST é um conjunto de dados de imagens de artigos de Zalando—consistindo em um conjunto de treinamento de 60.000 exemplos e um conjunto de teste de 10.000 amostras. Cada amostra é uma imagem em tons de cinza 28x28, associada a um rótulo de 10 classes. Zalando pretende que o Fashion-MNIST sirva como um substituto direto para o conjunto de dados MNIST original para benchmarking de algoritmos de aprendizado de máquina.

Ele compartilha o tamanho exato da imagem e a estrutura das divisões de treinamento e teste. O conjunto de dados MNIST original contém muitos dígitos manuscritos. Os membros da comunidade AI/ML/Data Science adoram esse conjunto de dados e o usam como referência para validar seus algoritmos. Na verdade, o MNIST é frequentemente o primeiro conjunto de dados que os pesquisadores tentam.

Cada imagem tem 28 pixels de altura e 28 pixels de largura, para um total de 784 pixels no total.
Cada pixel tem um único valor de pixel associado a ele, indicando a claridade ou escuridão daquele pixel, com números mais altos significando mais escuro. Este valor de pixel é um número inteiro entre 0 e 255.

Os conjuntos de dados de treinamento e teste têm 785 colunas. A primeira coluna é composta pelos rótulos das classes (veja acima), e representa a peça de vestuário. O restante das colunas contém os valores de pixel da imagem associada.

Para localizar um pixel na imagem, suponha que decompusemos x como x = i * 28 + j, onde i e j são inteiros entre 0 e 27. O pixel está localizado na linha i e coluna j de uma matriz 28 x 28 . Por exemplo, pixel31 indica o pixel que está na quarta coluna da esquerda e na segunda linha da parte superior, como no diagrama ascii abaixo.

## 3. Execução do experimento

A execução do experimento está dividida em etapas:

3.1 Preparar o conjunto de dados padrão;
3.2 Engenharia de requisitos;
3.3 Selecionar e treinar os modelos;
3.4 Avaliar os modelos;
3.5 Ajustar os modelos.

### 3.1 Preparando o conjunto de dados

Os modelos SVM e ConvNet não necessitam de modificações nos dados, podendo ser treinado com as imagens de tamanho 28x28 e um canal, porém, para transferência de conhecimento é necessário ajustá-las para no mínimo 48x48 com três canais (RGB).

#### 3.1.1 Coletando e rotulando os dados

O conjunto de dados Fashion do MNist já está rotulado e faz parte da biblioteca de conjuntos de dados do Keras.

Formato dos conjuntos de dados:

In [ ]:
train_X, train_Y, test_X, test_Y = load_mnist_dataset()

train_X.shape, train_Y.shape, test_X.shape, test_Y.shape

In [ ]:
test_ext_X, test_ext_Y = load_extra_dataset("../input/extra-fashion-test/test_images")

test_ext_X.shape, test_ext_Y.shape

#### 3.1.2 Avaliando os dados

In [ ]:
dataset_size = train_X.shape[0]+test_X.shape[0]
train_size = train_X.shape[0]
test_size = test_X.shape[0]
extra_test_size = test_ext_X.shape[0]

print(f"Train with {train_size:,} images that represents {round(train_size / dataset_size*100,2)}% of dataset size of {dataset_size:,}.")
print(f"Test with {test_size:,} images that represents {round(test_size / dataset_size*100,2)}% of dataset size of {dataset_size:,}.")
print(f"Test with extra {extra_test_size:,} images that represents {round(extra_test_size / dataset_size*100,2)}% of dataset size of {dataset_size:,}.")

Distribuição dos conjuntos de dados:

In [ ]:
train_data = pd.DataFrame(np.asarray(np.c_[train_Y, train_X.reshape(train_X.shape[0], 784)]))
test_data = pd.DataFrame(np.asarray(np.c_[test_Y, test_X.reshape(test_X.shape[0], 784)]))
test_ext_data = pd.DataFrame(np.asarray(np.c_[test_ext_Y, test_ext_X.reshape(test_ext_X.shape[0], 784)]))

print("--- Train data ---")
get_classes_distribution(train_data)
print("--- Test data ---")
get_classes_distribution(test_data)
print("--- Test with extra data ---")
get_classes_distribution(test_ext_data)

## 3.2 Engenharia de características 

Para o treinamento do modelo SVM foi aplicada a redução do número de caracteristicas utilizando HOG, já para os modelos pré treinados foi necessário aumentar esse conjunto, como demonstrado nas seções seguintes.

### 3.2.1 Preprocessar os dados

Para aplicar a transferência de conhecimento (_transfer learning_) para os modelos selecionado, é necessário:

- Converter as imagens em 3 canais para ajustá-las a entrada dos modelos;
- Redefinir o formato para o formato do _tensor_ (requerido pelo tensorflow);
- Redimensionar as imagens para 48x48.


In [ ]:
train_X = adjust_data_for_transfer_learning(train_X, 48)
test_X = adjust_data_for_transfer_learning(test_X, 48)
test_ext_X = adjust_data_for_transfer_learning(test_ext_X, 48)
print(f"train_X: {train_X.shape}")
print(f"test_X: {test_X.shape}")
print(f"test_ext_X: {test_ext_X.shape}")

Formato e exemplo de uma figura no conjunto de dados.

In [ ]:
exibe_bitmap_primeira_imagem(train_X)

### 3.2.2 Normalizar os dados

Os dados devem ser pré-processados antes de treinar a rede. Ao inspecionar a primeira imagem no conjunto de treinamento, verá que os valores de pixel ficam no intervalo de 0 a 255.

Escalamos esses valores para um intervalo de 0 a 1 antes de alimentá-los ao modelo de rede neural. Para isso, dividimos os valores por 255. É importante que o conjunto de treinamento e o conjunto de teste sejam pré-processados da mesma forma.

In [ ]:
# Normalize the data and change data type
train_X = train_X / 255.
test_X = test_X / 255.
test_ext_X = test_ext_X / 255.

Conjunto de dados normalizado:

In [ ]:
exibe_grade_imagens(AMOSTRAS_GRID, train_X, train_Y, must_reshape=False)

Conjunto extra de dados normalizado.

In [ ]:
exibe_grade_imagens(AMOSTRAS_GRID, test_ext_X, test_ext_Y, must_reshape=False)

### 3.2.3 Converter rótulos em codificador one-hot (para categórico)

Para variáveis categóricas em que não existe relacionamento ordinal, a codificação inteira pode não ser suficiente, na melhor das hipóteses, ou enganosa para o modelo, na pior.

Forçar uma relação ordinal por meio de uma codificação ordinal e permitir que o modelo assuma uma ordenação natural entre categorias pode resultar em desempenho ruim ou resultados inesperados.

Nesse caso, uma codificação _one-hot_ pode ser aplicada à representação ordinal. É aqui que a variável codificada de inteiro é removida e uma nova variável binária é adicionada para cada valor inteiro exclusivo na variável.

> Cada bit representa uma categoria possível. Se a variável não pode pertencer a várias categorias ao mesmo tempo, apenas um bit no grupo pode estar “ligado”. Isso é chamado de codificação one-hot.

In [ ]:
train_Y_one_hot = to_categorical(train_Y)
test_Y_one_hot = to_categorical(test_Y)
test_ext_Y_one_hot = to_categorical(test_ext_Y) 

In [ ]:
print("Examples:")
print(f"Category: {train_Y[0]}, Dummy vars: {train_Y_one_hot[0]}")
print(f"Category: {train_Y[1]}, Dummy vars: {train_Y_one_hot[1]}")

### 3.2.4 Dividindo os dados de treinamento em treinamento e validação

In [ ]:
train_X, valid_X, train_label, valid_label = train_test_split(train_X,
                                                              train_Y_one_hot,
                                                              test_size=0.05,
                                                              random_state=42)

## 3.3 Selecionar e treinar os modelos

Para transferência de aprendizado, foram escolhidos arbitrariamente três modelos com arquiteturas distintas do módulo de aplicativos do Keras. Eles são modelos de aprendizado profundo que são disponibilizados juntamente com pesos pré-treinados. Esses modelos podem ser usados para previsão, extração de recursos e ajuste fino.

Com base na avaliação do keras dos [modelo disponíveis](https://keras.io/api/applications/), selecionamos modelos com acurária (Top-5) acima de 90%.

Esses modelos são:

1. [DenseNet169](https://keras.io/api/applications/densenet/#densenet169-function), uma rede convolucionais densamente conectadas (HUANG et al., 2017)
2. [ResNet152V2](https://keras.io/api/applications/resnet/#resnet152v2-function), uma redes residuais profundas. (RE et al., 2016)
3. e a [VGG-16](https://keras.io/api/applications/vgg/#vgg16-function), uma dede convolucional muito profundas para reconhecimento de imagem em grande escala (SIMONYAN et al., 2014)

De acordo com essa [avaliação](https://keras.io/api/applications/), essas redes tem o seguinte desempenho:

| Modelo      | Tamanho (MB) | Top-1 Acurácia | Top-5 Acurácia | Paâmetros  | Profundidade |
| ---         | ---          | ---            | ---            | ---        | ---          |
| DenseNet169 | 57           | 76.2%          | 93.2%          | 14.3M      | 338          |
| ResNet152V2 | 232          | 78.0%          | 94.2%          | 60.4M      | 307          |
| VGG16       | 528          | 71.3%          | 90.1%          | 138.4M     | 16           |

> A profundidade conta o número de camadas com parâmetros.

O DenseNet-169 foi escolhido porque, apesar de ter uma profundidade de 169 camadas, é relativamente baixo em parâmetros em comparação com outros modelos, e a arquitetura lida bem com o problema do gradiente de fuga.

A escolha da ResNet é por causa da sua arquitetura, ela aprende com as funções residuais em vez de aprender com o sinal diretamente.

A VGG16 é a escolha preferida da comunidade para extrair recursos de imagens. A configuração de peso do VGGNet está disponível publicamente e tem sido usada em muitos outros aplicativos e desafios como um extrator de recursos de linha de base. (SIMONYAN et al., 2014)

### 3.3.1 Definindo os modelos para tranferência de conhecimento (Pre-training models)



Hiperparametros

In [ ]:
image_size = train_X[0].shape[0]
channels = 3
print("imageSize: ", image_size)

epochs = 30
batch_size = 700

## 3.4 Treinar modelo

### DenseNet169

In [ ]:
model_DenseNet169 = adj_model_DenseNet169(image_size, channels)

# Train
start = time.perf_counter()
history_DenseNet169_model = model_DenseNet169.fit(train_X, 
                                                train_label, 
                                                validation_data = (valid_X,
                                                                   valid_label), 
                                                epochs = epochs, 
                                                batch_size = batch_size, 
                                                verbose = 1)
end = time.perf_counter()
train_duration_DenseNet169 = end - start

Visualizando o modelo:

In [ ]:
plot_model(model_DenseNet169, 
           to_file = "plot-densenet169.png", 
           show_shapes = True, 
           show_layer_names = True)

### ResNet152V2

In [ ]:
model_ResNet152V2 = adj_model_ResNet152V2(image_size, channels)

# Train
start = time.perf_counter()
history_ResNet152V2_model = model_ResNet152V2.fit(train_X, 
                                                train_label, 
                                                validation_data = (valid_X,
                                                                    valid_label), 
                                                epochs = epochs, 
                                                batch_size = batch_size, 
                                                verbose = 1)
end = time.perf_counter()
train_duration_ResNet152V2 = end - start

In [ ]:
plot_model(model_ResNet152V2,
           to_file = "plot-resnet152v2.png", 
           show_shapes = True, 
           show_layer_names = True)

### VGG-16

In [ ]:
model_VGG16 = adj_model_VGG16(image_size, channels)

# Train
start = time.perf_counter()
history_VGG16_model = model_VGG16.fit(train_X, 
                                      train_label, 
                                      validation_data = (valid_X,
                                                        valid_label), 
                                      epochs = epochs,
                                      batch_size = batch_size, 
                                      verbose = 1)
end = time.perf_counter()
train_duration_VGG16 = end - start

In [ ]:
plot_model(model_VGG16,
           to_file = "plot-vgg16.png",
           show_shapes = True,
           show_layer_names = True)

## 3.4 Avaliar os modelos

A métrica de avaliação para os modelos será a precisão multiclasse.

### 3.4.1 Visualizando a acurácia e as perdas 

#### DenseNet169

In [ ]:
plot_acc_loss(history_DenseNet169_model, "DenseNet169", epochs);

#### ResNet152V2

In [ ]:
plot_acc_loss(history_ResNet152V2_model, "ResNet152V2", epochs);

#### VGG16

In [ ]:
plot_acc_loss(history_VGG16_model, "VGG16", epochs);

### 3.4.2 Testando os modelos

#### DenseNet169

In [ ]:
print("Avaliando DenseNet169")
print(model_DenseNet169.metrics_names)
me = model_DenseNet169.evaluate(test_X, test_Y_one_hot)
print(me)

In [ ]:
print("Avaliando DenseNet169")
print(model_DenseNet169.metrics_names)
me =model_DenseNet169.evaluate(test_ext_X, test_ext_Y_one_hot)
print(me)

 #### ResNet152V2

In [ ]:
print("Avaliando ResNet152V2")
print(model_ResNet152V2.metrics_names)
me = model_ResNet152V2.evaluate(test_X, test_Y_one_hot)
print(me)

In [ ]:
print("Avaliando ResNet152V2")
print(model_ResNet152V2.metrics_names)
me = model_ResNet152V2.evaluate(test_ext_X, test_ext_Y_one_hot)
print(me)

#### VGG16

In [ ]:
print("Avaliando VGG-16")
print(model_VGG16.metrics_names)
me = model_VGG16.evaluate(test_X, test_Y_one_hot)
print(me)

In [ ]:
print("Avaliando VGG-16")
print(model_VGG16.metrics_names)
me = model_VGG16.evaluate(test_ext_X, test_ext_Y_one_hot)
print(me)

### 3.4.3 Predições com os modelos

Avaliação das previsões em relação ao _Ground Truth_.

In [ ]:
# predict DenseNet169 Model
start = time.perf_counter()
pred_Y_DenseNet169 = model_DenseNet169.predict(test_X)
end = time.perf_counter()

show_predict(pred_Y_DenseNet169, test_X, test_Y, "DenseNet169")

predict_duration_DenseNet169 =  end - start

In [ ]:
# predict ResNet152V2 Model
start = time.perf_counter()
pred_Y_ResNet152V2 = model_ResNet152V2.predict(test_X)
end = time.perf_counter()

show_predict(pred_Y_ResNet152V2, test_X, test_Y, "ResNet152V2")

predict_duration_ResNet152V2 = predict_duration_DenseNet169

In [ ]:
# predict VGG16 Model
start = time.perf_counter()
pred_Y_VGG16 = model_VGG16.predict(test_X)
end = time.perf_counter()

show_predict(pred_Y_VGG16, test_X, test_Y, "VGG16")

predict_duration_VGG16 = end - start

Predição com conjunto extra de dados

In [ ]:
#predict DenseNet169 Model
pred_ext_Y_DenseNet169 = model_DenseNet169.predict(test_ext_X)
show_predict(pred_Y_DenseNet169, test_ext_X, test_ext_Y, "DenseNet169")

In [ ]:
#predict ResNet152V2 Model
pred_ext_Y_ResNet152V2 = model_ResNet152V2.predict(test_ext_X)
show_predict(pred_Y_ResNet152V2, test_ext_X, test_ext_Y, "ResNet152V2")

In [ ]:
#predict VGG16 Model
pred_ext_Y_VGG16 = model_VGG16.predict(test_ext_X)
show_predict(pred_Y_VGG16, test_ext_X, test_ext_Y, "VGG16")

### 3.4.4 Matriz de confusão para verificar a precisão

In [ ]:
# confusion matrix for DenseNet169 Model
show_confusion_matrix(test_Y, pred_Y_DenseNet169)

In [ ]:
# confusion matrix for ResNet152V2 Model
show_confusion_matrix(test_Y, pred_Y_ResNet152V2)

In [ ]:
# confusion matrix for VGG16 Model
show_confusion_matrix(test_Y, pred_Y_VGG16)

Matriz de confusão com conjunto extra de dados.

In [ ]:
# confusion matrix for DenseNet169 Model
show_confusion_matrix(test_ext_Y, pred_ext_Y_DenseNet169)

In [ ]:
# confusion matrix for ResNet152V2 Model
show_confusion_matrix(test_ext_Y, pred_ext_Y_ResNet152V2)

In [ ]:
# confusion matrix for VGG16 Model
show_confusion_matrix(test_ext_Y, pred_ext_Y_VGG16)

### 3.4.5 Relatório de classificação

In [ ]:
# Classification Report for DenseNet169 Model
cr = show_classification_report(test_Y, pred_Y_DenseNet169, CLASS_NAMES, "DenseNet169")

# Prepare report
df_model_metrics = df_model_metrics.append(add_model_metrics(cr, train_duration_DenseNet169, predict_duration_DenseNet169, "DenseNet169", 0))

In [ ]:
# Classification Report for ResNet152V2 Model
cr = show_classification_report(test_Y, pred_Y_ResNet152V2, CLASS_NAMES, "ResNet152V2")

# Prepare report
df_model_metrics = df_model_metrics.append(add_model_metrics(cr, train_duration_ResNet152V2, predict_duration_ResNet152V2, "ResNet152V2", 1))

In [ ]:
# Classification Report for DenseNet169 Model
cr = show_classification_report(test_Y, pred_Y_VGG16, CLASS_NAMES, "VGG16")

# Prepare report
df_model_metrics = df_model_metrics.append(add_model_metrics(cr, train_duration_VGG16, predict_duration_VGG16, "VGG16", 2))

Relatório de classificação com conjunto extra de dados.

In [ ]:
# Classification Report for DenseNet169 Model
cr = show_classification_report(test_ext_Y, pred_ext_Y_DenseNet169, CLASS_NAMES, "DenseNet169")

In [ ]:
# Classification Report for ResNet152V2 Model
cr = show_classification_report(test_ext_Y, pred_ext_Y_ResNet152V2, CLASS_NAMES, "ResNet152V2")

In [ ]:
# Classification Report for DenseNet169 Model
cr = show_classification_report(test_ext_Y, pred_ext_Y_VGG16, CLASS_NAMES, "VGG16")

## Conclusão

Os três modelos obtiveram um resultado muito ruim com o conjunto extra de dados, mas foram bem com os dados de testes. Dos três modelos o ResNet152V2 teve a melhor acurácia, porém com o maior custo de tempo para treinamento, cerca de quase o dobro do tempo da DenseNet169 e quase o triplo da VGG16, que tem os menores tempos de treinamento e seis vezes menos tempo para predição.

Em relação ao uso de GPU, todos os treinamentos utilizaram mais de 90% dos recursos da GPU.

![](https://github.com/asantos2000/ml-ipt-hw/raw/master/pics/uso-gpu.jpg)

In [ ]:
df_model_metrics

## Hardware

Os testes foram realizados no kaggle.com com a seguinte configuração:

- Aceleração: GPU T4 x 2
- Language python 3.7.10

O Kaggle executa os notebooks em [container docker](https://github.com/Kaggle/docker-python).

Em resumo o Kaggle disponibiliza o seguinte hardware:

| Hardware Component          | Release Year | Core Count      | Memory | Hours/Week |
| --------------------------- | ------------ | --------------- | ------ | ---------- |
| Intel Xeon CPU 2.00 GHz CPU | 2012         | 4 vCPU cores    | 18 GB  | Unlimited  |
| NVIDIA T4 (x2)              | 2018         | 2560 Cuda cores | 16 GB  | 33 h       | 

Mais informações sobre o hardware.

### CPU

In [ ]:
!lscpu | grep -vE "Vulnerability"

### GPU

In [ ]:
!nvidia-smi

## Referências

[^1]: ZHENG, Alice; CASARI, Amanda. Feature engineering for machine learning: principles and techniques for data scientists. " O'Reilly Media, Inc.", 2018.

HUANG, Gao et al. Densely connected convolutional networks. In: Proceedings of the IEEE conference on computer vision and pattern recognition. 2017. p. 4700-4708.

HE, Kaiming et al. Identity mappings in deep residual networks. In: European conference on computer vision. Springer, Cham, 2016. p. 630-645.

SIMONYAN, Karen; ZISSERMAN, Andrew. Very deep convolutional networks for large-scale image recognition. arXiv preprint arXiv:1409.1556, 2014.

